# Part 1

In [ ]:
test_data = """$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k""".splitlines()
test_data = [line.strip() for line in test_data]
test_data[:4]

In [ ]:
with open("../07.in") as f:
    data = f.readlines()
    data = [line.strip() for line in data]
data[:4]

In [ ]:
from typing import Any

class Directory():
    def __init__(self, parent: Any, name: str) -> None:
        self.parent = parent
        self.name = name
        self.subdirs = []
        self.files: set[tuple[str, int]] = set()
    
    def get_size(self) -> int:
        file_size = sum((f[1] for f in self.files))
        folder_size = sum(d.get_size() for d in self.subdirs)
        return file_size + folder_size
    
    def get_full_name(self):
        if self.name == "/" or not self.parent:
            return self.name
        else:
            return self.parent.get_full_name() + "/" + self.name

    def aggrogate_sizes(self):
        result = {self.get_full_name(): self.get_size()}
        for subdir in self.subdirs:
            result.update(subdir.aggrogate_sizes())
        return result

    def cd(self, target: str):
        if target == "..":
            return self.parent
        
        for subdir in self.subdirs:
            if subdir.name == target:
                return subdir
        else:
            subdir = Directory(self, target)
            self.subdirs.append(subdir)
            return subdir

    def print(self, depth: int=0):
        print((depth-1)*"\t", self.name, self.get_size())
        for file in self.files:
            print(depth*"\t", file[0], file[1])
        for dir in self.subdirs:
            dir.print(depth+1)

In [ ]:
import re 

cmd_pattern = re.compile(r"\$ (\w+) ?(\D*)?")
file_pattern = re.compile(r"(\d*) (\w+(\.?\w+)*)")

root = Directory(None, "invisible root")

current = root
for line in data:
    match = cmd_pattern.match(line)
    if match:
        if match.group(1) == "cd":
            target = match.group(2)
            current = current.cd(target)
            continue
        else:
            # ls is the only other command,
            # but we dont have to handle it soooo
            continue
    
    match = file_pattern.match(line)
    if match: 
        size = int(match.group(1))
        name = match.group(2)
        current.files.add((name, size))
    else:
        # This should be a line starting with "dir"
        # which we will just ignore.
        pass

root.print()

In [ ]:
result = root.aggrogate_sizes()
filtered = list(filter(lambda x: x <= 100_000, result.values()))
sum(filtered)

# Part 2


In [ ]:
total_disk_size = 70_000_000
update_size = 30_000_000
used_disk_size = root.get_size()
to_free = update_size - (total_disk_size - used_disk_size)
print(f"Need to delete {to_free} from disk")

In [ ]:
result = root.aggrogate_sizes()
filtered = list(filter(lambda x: x >= to_free, result.values()))
sorted(filtered)[0]